In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import  json

class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('movie.jl', 'w')
        
    def close_spider(self, spider):
        self.file.close()
        
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + '\n'
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = 'movie'
    start_urls = ['https://imdb.com/search/title/?companies=fox&view=advanced']
    
    def pagination(self,response):
        page = response.css('div.desc a::attr(href)').extract_first()
        if page:
            pagination = repsonse.urljoin(page)
            yield scrapy.Request(url = url, callback=self.parse)

    custom_settings = {
        'LOG_LEVEL':logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': 'movie.json'
    }
    
    
    def parse(self, response):
        urls = response.css('h3.lister-item-header a::attr(href)').extract()
        for url in urls:
            url = response.urljoin(url)
            yield scrapy.Request(url = url, callback=self.parse_detail)
    
    
    def parse_detail(self,movie):
            yield {
                'judul': movie.css('div.title_wrapper h1::text').extract_first().replace('\u00a0',''),
                'genre': movie.css('div.subtext a::text').extract(),
                'bintang': movie.css('div.credit_summary_item a::text').extract(),
                'certificate': movie.css('div.titleBar div.subtext::text').extract_first().replace('\n                    ','').replace('\n    ',''),
                'storyline': movie.css('div.plot_summary div::text').extract_first().replace('\n                    ','').replace('\n            ','')
            }

In [4]:
process = CrawlerProcess({
    'USER_AGENT':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
})

process.crawl(QuotesSpider)
process.start()

2021-03-27 17:32:03 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-27 17:32:03 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-03-27 17:32:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-27 17:32:03 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
2021-03-27 17:32:03 [py.warnings] WARNING: C:\Users\NT\anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see 

In [5]:
!dir movie.*


 Volume in drive D is Data
 Volume Serial Number is 3673-85B8

 Directory of D:\Kuliah\Data Mining\DATA

27/03/2021  17:32            21.750 movie.jl
27/03/2021  17:32            43.504 movie.json
               2 File(s)         65.254 bytes
               0 Dir(s)  209.306.566.656 bytes free


In [6]:
!movie.json